## Get the data from the kaggle dataset using their API
Source: https://www.kaggle.com/datasets/andrewsundberg/college-basketball-dataset/data?select=cbb.csv

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import ticker

file_path = "cbb.csv"

lf = kagglehub.dataset_load(
  KaggleDatasetAdapter.POLARS,
  "andrewsundberg/college-basketball-dataset",
  file_path,
  # See the documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpolars
)

df = lf.collect()

## Exploratory Analysis
* There is a good amount of information about the dataset and column specifics on the kaggle page where the data is from

In [ ]:
df.head()

In [ ]:
df.describe()
## there are no nulls
## postseason and seed are strings which we may want to make numbers or binary later on
## the 2025 data is not included yet, we could add that in later if we really wanted to

## Data transformation:
* For this project, I am going to try to predict the number of postseason wins for each team in hopes of predicting the elusive perfect bracket
* This will require some transformations to the data namely:
    * translating the POSTSEASON column into a POSTSEASON_WINS column which will become our depenedent variable
    * removing all of the data for the teams that did not make the post-season: all of these teams would effectively have 0 post-season wins and would add a lot of data that has little value because we are aiming to differentiate between the top teams that made the tournament when completing a bracket

In [ ]:
## Looking at these results, we can figure out which variables we need to translate
print(df.get_column('POSTSEASON').unique().to_list())

## This output shows there is no differentiation for the Round of R68 teams
## Since these games are often ignored in classic 64 team brackets, they will be removed from the data
print(df.get_column('SEED').unique().to_list())
print('!!! CHECK THIS IS WHAT WE ENDED UP DOING !!!')

In [ ]:
POSTSEASON_to_WINS = {
    'R68':      0, 
    'R64':      0, 
    'R32':      1, 
    'S16':      2, 
    'E8':       3, 
    'F4':       4, 
    '2ND':      5, 
    'Champions':6, 
}

df_postseason_wins = (df
                      .filter(pl.col('POSTSEASON').is_in(POSTSEASON_to_WINS.keys()))
                      .with_columns(pl.col('POSTSEASON').replace(POSTSEASON_to_WINS).alias('POSTSEASON_WINS').cast(pl.Int32),
                                    pl.col('SEED').cast(pl.Int32))
                )

In [ ]:
df_postseason_wins

## EDA for the Postseason Wins dataframe

#### Total number of postseason wins by seed

In [ ]:
## data prep
postseason_wins_by_seed = (df_postseason_wins
        .group_by(pl.col('SEED')).agg(pl.col('POSTSEASON_WINS').sum())
        .sort('SEED')
)

## plot
fig, ax = plt.subplots(figsize = (8,5))

plt.bar(postseason_wins_by_seed['SEED'], postseason_wins_by_seed['POSTSEASON_WINS'])

ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
plt.xlim(postseason_wins_by_seed['SEED'].min()-.5, postseason_wins_by_seed['SEED'].max()+.5)

for bars in ax.containers:
    ax.bar_label(bars)
    
plt.xlabel('Seed'), plt.ylabel('Wins');

A few interesting results intitially:
* 1 seeds are far and away the most winning teams, no surprises there
* There have been some standout 11 seeds with nearly as many wins as 5 seeds!

#### Integrity check: make sure there are the same number of wins per year

In [ ]:
## This should just be a quick check that every year has the same number of total wins in the tourney (63)

## data prep
wins_by_year = (df_postseason_wins
        .group_by(pl.col('YEAR')).agg(pl.col('POSTSEASON_WINS').sum())
        .sort('YEAR')
)

## plot
fig, ax = plt.subplots(figsize = (8,5))

plt.bar(wins_by_year['YEAR'], wins_by_year['POSTSEASON_WINS'])

ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
plt.xlim(wins_by_year['YEAR'].min()-.5, wins_by_year['YEAR'].max()+.5)

for bars in ax.containers:
    ax.bar_label(bars)
    
plt.xlabel('Year'), plt.ylabel('Wins');

#### Total wins by team since 2013 (min. 10 wins)

In [ ]:
## data prep
postseason_wins_by_team = (df_postseason_wins
        .group_by(pl.col('TEAM')).agg(pl.col('POSTSEASON_WINS').sum())
        .filter(pl.col('POSTSEASON_WINS') >= 10)
        .sort('POSTSEASON_WINS')
)

## plot
fig, ax = plt.subplots(figsize = (8,5))

plt.barh(postseason_wins_by_team['TEAM'], postseason_wins_by_team['POSTSEASON_WINS'])

ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    
plt.xlabel('Wins')